Importing everything we need

In [68]:
import os
import csv
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from astrapy import DataAPIClient
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

Load the env variables and declare the dataset path

In [69]:
load_dotenv()

# Retrieve Astra DB credentials and settings from environment variables
ASTRA_TOKEN = os.getenv("ASTRA_TOKEN")
ASTRA_DB_URL = os.getenv("ASTRA_DB_URL")
COLLECTION_NAME = os.getenv("COLLECTION_NAME")
file_path="./dataset.csv"

Now, we connect to the DB


In [70]:
client = DataAPIClient(ASTRA_TOKEN)
db = client.get_database_by_api_endpoint(ASTRA_DB_URL)

If this is the First time, Then we have to create the new collection. creating a new collection if it does not exist already

In [62]:
if COLLECTION_NAME not in db.list_collection_names():
    db.create_collection(COLLECTION_NAME)
    print("Created Collection")
else:
    print("Collection already exists")

Collection already exists


Now, we read the rows from the dataset and then feed it to the database one by one

In [55]:
collection = db.get_collection(COLLECTION_NAME)
try:
    with open(file_path, mode='r') as file:
        csv_reader = csv.DictReader(file)
        c=1
        for row in csv_reader:
            # Prepare document to insert into the collection
            document = {
                "post_id": row["post_id"],
                "day_of_posting": row["day_of_posting"],
                "date_of_posting": row["date_of_posting"],
                "time_of_posting": row["time_of_posting"],
                "post_type": row["post_type"],
                "likes": int(row["likes"]),
                "comments": int(row["comments"]),
                "shares": int(row["shares"]),
                "repost": int(row["repost"]),
                "gender": row["gender"],
                "hashtags": row["hashtags"].split(",") if row["hashtags"].strip() else []
            }

            # Insert into the collection
            collection.insert_one(document)
            print("Processed row {}".format(c))
            c+=1

        print("Data successfully inserted into the database.")
except FileNotFoundError:
    print(f"Error: File '{file_path}' not found.")
except KeyError as e:
    print(f"Error: Missing required column in the file: {e}")
except ValueError as e:
    print(f"Error: Invalid data format: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

Processed row 1
Processed row 2
Processed row 3
Processed row 4
Processed row 5
Processed row 6
Processed row 7
Processed row 8
Processed row 9
Processed row 10
Processed row 11
Processed row 12
Processed row 13
Processed row 14
Processed row 15
Processed row 16
Processed row 17
Processed row 18
Processed row 19
Processed row 20
Processed row 21
Processed row 22
Processed row 23
Processed row 24
Processed row 25
Processed row 26
Processed row 27
Processed row 28
Processed row 29
Processed row 30
Processed row 31
Processed row 32
Processed row 33
Processed row 34
Processed row 35
Processed row 36
Processed row 37
Processed row 38
Processed row 39
Processed row 40
Processed row 41
Processed row 42
Processed row 43
Processed row 44
Processed row 45
Processed row 46
Processed row 47
Processed row 48
Processed row 49
Processed row 50
Processed row 51
Processed row 52
Processed row 53
Processed row 54
Processed row 55
Processed row 56
Processed row 57
Processed row 58
Processed row 59
Proces

### Lets check what is inside the DB

In [71]:
try:
    data = collection.find()
    data= [doc for doc in data]
    print(data)
except Exception as e:
    print(f"Error retrieving data from Astra DB: {e}")

[{'_id': '081f9214-a5c2-419d-9f92-14a5c2b19dee', 'post_id': '0000000000000086', 'day_of_posting': 'Tuesday', 'date_of_posting': '2024-10-07', 'time_of_posting': '14:15:00', 'post_type': 'reels', 'likes': 240, 'comments': 55, 'shares': 30, 'repost': 15, 'gender': 'M', 'hashtags': ['#Tech']}, {'_id': '5b410d66-545a-462a-810d-66545aa62a3f', 'post_id': '0000000000000007', 'day_of_posting': 'Sunday', 'date_of_posting': '2024-12-25', 'time_of_posting': '13:30:00', 'post_type': 'reels', 'likes': 270, 'comments': 60, 'shares': 35, 'repost': 18, 'gender': 'M', 'hashtags': ['#Food']}, {'_id': '3afb44de-ffad-4ba9-bb44-deffad6ba983', 'post_id': '0000000000000032', 'day_of_posting': 'Thursday', 'date_of_posting': '2024-11-30', 'time_of_posting': '13:30:00', 'post_type': 'reels', 'likes': 250, 'comments': 60, 'shares': 35, 'repost': 18, 'gender': 'F', 'hashtags': ['#Vlog']}, {'_id': '92b77a46-1d63-4d39-b77a-461d63cd396b', 'post_id': '0000000000000099', 'day_of_posting': 'Monday', 'date_of_posting': 

# if we use OpenAI's LLM

In [ ]:
llm = OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"))
prompt_template = (
    "Here is the entire dataset: {data}. "
    "Identify patterns, trends, and insights from this dataset. "
    "Summarize the key trends only without providing row-by-row details."
    "For example, One trend could be 'Posts by females receive 46% more likes compared to posts by male'. similarly generate more insights from the data and use Numbers to further strengthen the trends."
    "Finally, the output should be a list of trends and insights from the data. do not genererate anything else, no suggestions."
    "Generate as many trends as possible, including all columns of the data, except the _id and post_id."
    "For example, show the correlation between posts at different times and the engagement"
)
prompt = PromptTemplate(input_variables=["data"], template=prompt_template)
chain = (
    {"data": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


# If we use Gemini 1.5 Flash

In [72]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=os.getenv("GEMINI_API_KEY"))
prompt_template = (
    "Here is the entire dataset: {data}. "
    "Identify patterns, trends, and insights from this dataset. "
    "Summarize the key trends only without providing row-by-row details."
    "For example, One trend could be 'Posts by females receive 46% more likes compared to posts by male'. similarly generate more insights from the data and use Numbers to further strengthen the trends."
    "Finally, the output should be a list of trends and insights from the data. do not genererate anything else, no suggestions."
    "Generate as many trends as possible, including all columns of the data, except the _id and post_id."
    "For example, show the correlation between posts at different times and the engagement"
)
prompt = PromptTemplate(input_variables=["data"], template=prompt_template)
chain = (
    {"data": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [73]:
trends = chain.invoke(str(data))
print(trends)

* Reels receive significantly more likes (average 235) than other post types (average 120).
* Posts with hashtags receive more engagement (likes, comments, shares, and reposts) than those without.
*  Posts made on Tuesdays and Wednesdays tend to have higher engagement than other days.
*  There is a positive correlation between the number of likes and comments, shares, and reposts.
*  Male users tend to post more reels and receive slightly more likes on average than female users.
* The time of posting shows some influence on engagement, with posts between 12pm and 4pm generally receiving higher engagement.
* "#Food" and "#Tech" are the most frequently used hashtags, suggesting strong user interest in these topics.
*  "Mixed-carousel" and "video-carousel" post types have similar average engagement levels.
*  Static image posts receive the lowest average engagement.


